<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Sample notebook to build custom images from the lake user teamspace.

## Content
1. [Configuration](#Configuration)
2. [Build Image](#Build-Image)
2. [Running Container using the profile](#Running-container)


### Configuration

In [1]:
image_name = 'jupyter-with-apps'
folder_name = '/efs/shared/samples/images/extra_user_apps'

In [2]:
%cd $folder_name

/efs/shared/samples/images/extra_user_apps


### Build Image

Lets see how  orbit build image works...

get our orbit env and team names

In [3]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

('test-env-public', 'lake-creator')

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}-users-'

In [ ]:
repository_name = (f"orbit-{env_name}-users-{image_name}")
repository_name

In [ ]:
!aws ecr delete-repository --repository-name $repository_name --force

In [ ]:
%cd /samples/images/images/extra_user_apps

adding a file to our docker as an example

In [ ]:
pwd = %pwd
pwd

Now lets run the command

In [ ]:
%%time

output = !orbit build image -e dev-env -d $pwd -n $image_name
output

Lets get the image address from the output of the previous command

In [ ]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

In [ ]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

### Building the profile for the Image

In [4]:
import json
profile = {
        "display_name": 'Workbench Apps (mid-size)',
        "description": "Use simple custom image",
        "kubespawner_override": {
            "image": image,
#             "image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-user-apps",
            "cpu_guarantee": 4,
            "cpu_limit": 4,
            "mem_guarantee": "4G",
            "mem_limit": "4G"
        }
}

with open("profile.json", 'w') as f:
    json.dump(profile, f)


In [5]:
!cat profile.json

{"display_name": "Workbench Apps (mid-size)", "description": "Use simple custom image", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-user-apps", "cpu_guarantee": 4, "cpu_limit": 4, "mem_guarantee": "4G", "mem_limit": "4G"}}

In [8]:
!orbit build profile --debug --env $env_name --team lake-user profile.json

[2021-04-01 16:21:09,442][__main__.py  :481] env: test-env-public
[2021-04-01 16:21:09,442][__main__.py  :482] team: lake-user
[2021-04-01 16:21:09,442][__main__.py  :484] profile: {"display_name": "Workbench Apps (mid-size)", "description": "Use simple custom image", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-user-apps", "cpu_guarantee": 4, "cpu_limit": 4, "mem_guarantee": "4G", "mem_limit": "4G"}}
[2021-04-01 16:21:09,442][__main__.py  :485] debug: True
[2021-04-01 16:21:09,442][messages.py  :133] Progress bar: 1%
[ Info ] Retrieving existing profiles
[2021-04-01 16:21:09,443][image.py     : 35] Trying to read profiles from SSM parameter (/orbit/test-env-public/teams/lake-user/user/profiles).
[2021-04-01 16:21:09,600][image.py     :102] Existing user profiles for team lake-user: []
[2021-04-01 16:21:09,600][image.py     :116] new profile name: Workbench Apps (mid-size)
[ Tip ] Profile added Workbench Apps (mid-s

### Running container 

Lets run a container using the profile and image we created

In [ ]:
import json
run = {
      "compute": {
          "container" : {
              "p_concurrent": "1"
          },
          "node_type": "ec2",
          "profile": profile_name
      },
      "tasks":  [{
          "notebookName": "test-image.ipynb",
          "sourcePath": pwd,
          "targetPath": f"/efs/shared/regression/notebooks/I-Image/{folder_name}",
          "params": {
          }
        }]
 }

with open("run.json", 'w') as f:
    json.dump(run, f)


In [ ]:
%%time

!orbit run notebook --env $env_name --team $team_name --user testing --wait --tail-logs run.json
